In [ ]:
import torch
print(torch.__version__)
print(torch.cuda.is_available())
print(torch.version.cuda)


In [3]:
import os

dataset_path = "F:/fenlei/train"
print(os.listdir(dataset_path))  # 列出数据集中顶层文件夹

from torchvision import datasets, transforms

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
])

dataset = datasets.ImageFolder(root=dataset_path, transform=transform)

# 打印类别和样本数量
print(f"Classes: {dataset.classes}")  # 列出类别名称
print(f"Class-to-Index Mapping: {dataset.class_to_idx}")  # 类别到索引的映射
print(f"Total Images: {len(dataset)}")  # 数据集中图片总数


['Harmful', 'Kitchen', 'Other', 'Recyclable']
Classes: ['Harmful', 'Kitchen', 'Other', 'Recyclable']
Class-to-Index Mapping: {'Harmful': 0, 'Kitchen': 1, 'Other': 2, 'Recyclable': 3}
Total Images: 64000


In [6]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms
from torchvision.models import resnet18, ResNet18_Weights
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

# 数据预处理
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
])

# 数据集加载和划分
dataset_path = "F:/fenlei/train"
dataset = datasets.ImageFolder(root=dataset_path, transform=transform)
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_set, val_set = random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_set, batch_size=32, shuffle=True)
val_loader = DataLoader(val_set, batch_size=32, shuffle=False)

# 加载预训练模型并修改最后一层
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = resnet18(weights=ResNet18_Weights.DEFAULT)
model.fc = nn.Linear(model.fc.in_features, 4)  # 修改为 4 类输出
model = model.to(device)

# 损失函数和优化器
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# 训练循环
epochs = 2
for epoch in range(epochs):
    model.train()
    train_loss, correct = 0, 0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        # 前向传播
        outputs = model(images)
        loss = criterion(outputs, labels)

        # 反向传播与优化
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # 统计损失和准确率
        train_loss += loss.item() * images.size(0)
        _, preds = torch.max(outputs, 1)
        correct += (preds == labels).sum().item()

    print(f"Epoch [{epoch+1}/{epochs}], Loss: {train_loss/len(train_set):.4f}, Acc: {correct/len(train_set):.4f}")



Epoch [1/2], Loss: 0.2543, Acc: 0.9086


KeyboardInterrupt: 

In [35]:
import os
from PIL import Image
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import torch

# 自定义未标记数据集类
class UnlabeledDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.image_paths = [os.path.join(root_dir, img) for img in os.listdir(root_dir) if img.endswith(('.jpg', '.png', '.jpeg'))]

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert("RGB")
        if self.transform:
            image = self.transform(image)
        return image, img_path

# 路径和预处理
test_dataset_path = "F:/fenlei/eval"
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
])

# 创建测试数据加载器
test_dataset = UnlabeledDataset(root_dir=test_dataset_path, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# 模型评估和预测
model.eval()
predictions = []
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

with torch.no_grad():
    for images, image_paths in test_loader:
        images = images.to(device)
        outputs = model(images)
        _, preds = torch.max(outputs, 1)
        for img_path, pred in zip(image_paths, preds.cpu().numpy()):
            predictions.append((img_path, pred))

from collections import Counter

# 假设 predictions 是 (img_path, pred) 的列表
# 提取所有的预测类别
predicted_classes = [pred for _, pred in predictions]

# 统计每个类别的数量
class_counts = Counter(predicted_classes)

# 打印每个类别的统计结果
for class_id, count in class_counts.items():
    print(f"Class {class_id}: {count} images")



Class 1: 1221 images
Class 3: 1333 images
Class 2: 1075 images
Class 0: 1371 images


In [36]:
# 保存模型
torch.save(model.state_dict(), "./model.pth")

In [37]:
# 导出为 ONNX
dummy_input = torch.randn(1, 3, 224, 224).to(device)  # 模拟输入
torch.onnx.export(
    model,
    dummy_input,
    "./model.onnx",
    input_names=["input"],
    output_names=["output"],
    opset_version=11  # 使用适配 RDK X5 的 ONNX Opset 版本
)
import onnx

# 加载 ONNX 模型
onnx_model = onnx.load("./model.onnx")
onnx.checker.check_model(onnx_model)  # 检查模型是否有效
print("ONNX Model is valid.")


ONNX Model is valid.
